## Xception model training

### import libs

In [1]:
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
import tensorflow as tf

Using TensorFlow backend.


In [2]:
nb_classes = 20
nb_gpus = 4
image_size = (299, 299)
input_shape= (299,299,3)

train_path = "/home/cnn/Documents/tct/conference/xception/batch6_cells_half299/train/"
valid_path = "/home/cnn/Documents/tct/conference/xception/batch6_cells_half299/valid/"

### pretrain model

In [3]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights='imagenet', include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(0.5)(p_out)
    predictions = Dense(nb_classes, activation='softmax', name="predictions")(p_out)

    for layer in base_model.layers:
        layer.trainable = False

    model = Model(inputs=base_model.input, outputs=predictions)

if nb_gpus > 1:
    parallel_model = multi_gpu_model(model, gpus=nb_gpus)
parallel_model.compile(optimizer='Adadelta', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

In [4]:
batch_size = 64
epochs = 1

In [5]:
img_gen_t = ImageDataGenerator()
train_generator = img_gen_t.flow_from_directory(train_path, 
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory(valid_path,
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=len(train_generator), 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=len(valid_generator))

model.save_weights("Xception_first_train.h5")

Found 114939 images belonging to 20 classes.
Found 12762 images belonging to 20 classes.
Epoch 1/1
1796/1796 [==============================] - 574s 320ms/step - loss: 1.3318 - acc: 0.6063 - val_loss: 2.3916 - val_acc: 0.3245


### train model

In [3]:
input_tensor = Input(input_shape)
x = Lambda(xception.preprocess_input)(input_tensor)

base_model = Xception(input_tensor=x, weights=None, include_top=False)
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(0.5)(p_out)
predictions = Dense(nb_classes, activation='softmax', name="predictions")(p_out)

model = Model(inputs=base_model.input, outputs=predictions)

if nb_gpus > 1:
    model = multi_gpu_model(model, gpus=nb_gpus)
model.compile(optimizer='Adadelta', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_5 (

In [4]:
model.load_weights('weights_026_0.1794.hdf5')

In [5]:
batch_size = 64
epochs = 100

In [6]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard

img_gen_t = ImageDataGenerator(rotation_range=90,                            
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               zoom_range=0.1,
                               brightness_range=[0.9, 1.1],
#                               zca_whitening= True,
                               horizontal_flip=True,
                               vertical_flip=True)
train_generator = img_gen_t.flow_from_directory(train_path, 
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory(valid_path,
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

checkpoint = ModelCheckpoint("weights_{epoch:03d}_{val_loss:.4f}.hdf5", monitor='val_loss', verbose=1,
                             save_best_only=False, save_weights_only=True, mode='min', period=1)

tensorboard = TensorBoard(log_dir="./logs", histogram_freq=0, batch_size=batch_size, write_graph=True, write_images=True)

callbacks = [checkpoint, tensorboard]

model.fit_generator(generator=train_generator, 
                    steps_per_epoch=len(train_generator), 
                    epochs=epochs, 
                    verbose=1,
                    validation_data=valid_generator, 
                    validation_steps=len(valid_generator), 
                    callbacks=callbacks, 
                    workers=16, 
                    use_multiprocessing=True,
                    initial_epoch=26)

Found 114939 images belonging to 20 classes.
Found 12762 images belonging to 20 classes.
Epoch 27/100
1796/1796 [==============================] - 1166s 649ms/step - loss: 0.1222 - acc: 0.9537 - val_loss: 0.1824 - val_acc: 0.9371

Epoch 00027: saving model to weights_027_0.1824.hdf5
Epoch 28/100
1796/1796 [==============================] - 1129s 629ms/step - loss: 0.1179 - acc: 0.9551 - val_loss: 0.1745 - val_acc: 0.9368

Epoch 00028: saving model to weights_028_0.1745.hdf5
Epoch 29/100
1796/1796 [==============================] - 1130s 629ms/step - loss: 0.1193 - acc: 0.9553 - val_loss: 0.1933 - val_acc: 0.9372

Epoch 00029: saving model to weights_029_0.1933.hdf5
Epoch 30/100
1796/1796 [==============================] - 1136s 633ms/step - loss: 0.1172 - acc: 0.9553 - val_loss: 0.1835 - val_acc: 0.9368

Epoch 00030: saving model to weights_030_0.1835.hdf5
Epoch 31/100
1796/1796 [==============================] - 1148s 639ms/step - loss: 0.1140 - acc: 0.9564 - val_loss: 0.1699 - val_ac


Epoch 00068: saving model to weights_068_0.2296.hdf5
Epoch 69/100
1796/1796 [==============================] - 1150s 640ms/step - loss: 0.0694 - acc: 0.9742 - val_loss: 0.1863 - val_acc: 0.9446

Epoch 00069: saving model to weights_069_0.1863.hdf5
Epoch 70/100
1796/1796 [==============================] - 1140s 635ms/step - loss: 0.0705 - acc: 0.9738 - val_loss: 0.2445 - val_acc: 0.9355

Epoch 00070: saving model to weights_070_0.2445.hdf5
Epoch 71/100
1796/1796 [==============================] - 1156s 643ms/step - loss: 0.0675 - acc: 0.9751 - val_loss: 0.2536 - val_acc: 0.9309

Epoch 00071: saving model to weights_071_0.2536.hdf5
Epoch 72/100
1796/1796 [==============================] - 1151s 641ms/step - loss: 0.0675 - acc: 0.9750 - val_loss: 0.2414 - val_acc: 0.9388

Epoch 00072: saving model to weights_072_0.2414.hdf5
Epoch 73/100
1796/1796 [==============================] - 1136s 632ms/step - loss: 0.0677 - acc: 0.9754 - val_loss: 0.2301 - val_acc: 0.9396

Epoch 00073: saving mode

#### plot training trend

In [ ]:
# 1. first need to retrieve acc/loss/val_acc/val_loss info from tensorboard
# in terminal: tensorboard --logdir ./logs
# open browser with given link
# save data to local csv and merge four separate files

# 2. read from saved data csv
import pandas as pd
import matplotlib.pyplot as plt

csv_file = "./info_20181118.csv"

df = pd.read_csv(csv_file)

step = list(df.step)
acc = list(df.acc)
loss = list(df.loss)
val_acc = list(df.val_acc)
val_loss = list(df.val_loss)

# 3. plot acc and loss
from matplotlib.patches import Rectangle

fig = plt.figure(1, figsize=(12,6), dpi=90)
ax = fig.add_subplot(121)
ax.plot(step, acc,label="train")
ax.plot(step, val_acc,label="valid")
ax.legend()
plt.title("training accuracy")
plt.xlabel("epoch")
plt.ylabel("accuracy")
ax = fig.add_subplot(122)
ax.plot(step, loss,label="train")
ax.plot(step, val_loss,label="vaild")
ax.legend()
plt.title("training loss")
plt.xlabel("epoch")
plt.ylabel("loss")